In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from geopy.geocoders import Nominatim

In [5]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [6]:
location = geolocator.reverse("40.753808 -73.978325")

In [10]:
print(location.address)

335 Madison Avenue, 335, Madison Avenue, Midtown East, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10017, USA


In [12]:
print(location.raw)

{'place_id': '138371945', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': '266010414', 'lat': '40.75362205', 'lon': '-73.9780733316277', 'display_name': '335 Madison Avenue, 335, Madison Avenue, Midtown East, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10017, USA', 'address': {'building': '335 Madison Avenue', 'house_number': '335', 'road': 'Madison Avenue', 'suburb': 'Midtown East', 'city_district': 'Manhattan', 'city': 'NYC', 'county': 'New York County', 'state': 'New York', 'postcode': '10017', 'country': 'USA', 'country_code': 'us'}, 'boundingbox': ['40.7532159', '40.7539757', '-73.978603', '-73.9775817']}


In [14]:
#importing data
train = pd.read_csv("C:/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/generated_datasets/clean_2.csv")

In [15]:
train['pickup_latitude'] = train['pickup_latitude'].round(6)
train['pickup_longitude'] = train['pickup_longitude'].round(6)
train['dropoff_latitude'] = train['dropoff_latitude'].round(6)
train['dropoff_longitude'] = train['dropoff_longitude'].round(6)

In [23]:
#removing lat and long with 0.0 values
train = train[train['pickup_latitude'] != 0.0]
train = train[train['pickup_longitude'] != 0.0]
train = train[train['dropoff_latitude'] != 0.0]
train = train[train['dropoff_longitude'] != 0.0]

In [24]:
#pickup lat and long
train['pic_lat_long'] = train['pickup_latitude'].astype(str)+ ' ' + train['pickup_longitude'].astype(str)

#drop lat and long
train['drop_lat_long'] = train['dropoff_latitude'].astype(str)+ ' ' + train['dropoff_longitude'].astype(str)

In [49]:
train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day_of_month,date,day_of_week,weekend_weekday,hour,time_30_min_bin,time_15_min_bin,pic_lat_long,drop_lat_long,pic_address
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,3,10,2015-03-10,Tuesday,0,7,30,45,40.751041 -73.994102,40.754959 -73.991257,"Chelsea, Manhattan, Manhattan Community Board ..."
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,17,2011-12-17,Saturday,1,0,30,45,40.713017 -73.961686,40.699525 -73.952406,"152, South 2nd Street, Williamsburg, Kings Cou..."
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,9,16,2010-09-16,Thursday,0,0,0,15,40.753808 -73.978325,40.756177 -73.985068,"335 Madison Avenue, 335, Madison Avenue, Midto..."
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,6,13,2012-06-13,Wednesday,0,18,0,0,40.719357 -74.001737,40.740713 -73.992353,"The Oltarsh Building, Canal Street, Chinatown,..."
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,2,4,2015-02-04,Wednesday,0,7,30,30,40.770531 -73.966393,40.781441 -73.958328,"860, Madison Avenue, Upper East Side, Manhatta..."


In [26]:
train.dtypes

fare_amount          float64
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
year                   int64
month                  int64
day_of_month           int64
date                  object
day_of_week           object
weekend_weekday        int64
hour                   int64
time_30_min_bin        int64
time_15_min_bin        int64
pic_lat_long          object
drop_lat_long         object
dtype: object

In [28]:
def geo_loc(x):
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    location = geolocator.reverse(x)
    return location.address


In [35]:
#train['pic_address'] = train.pic_lat_long.apply(lambda x: geo_loc(x.decode('utf-8')))
train['pic_address'] = train.pic_lat_long.apply(lambda x: geo_loc(x))

In [54]:
train['drop_address'] = train.drop_lat_long.apply(lambda x: geo_loc(x))

GeocoderTimedOut: Service timed out

In [ ]:
train.to_csv('clean_4_with_geoLoc.csv', index=False )

In [ ]:
 train.pic_lat_long.apply(lambda x: geolocator.reverse(x)).head()